### Importing necessary libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
stpwrds=stopwords.words("english")
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModelForTableQuestionAnswering,pipeline, AutoTokenizer
import torch


2024-05-23 16:18:02.863184: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-23 16:18:03.958941: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/runcandel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/runcandel/anaconda3/envs/yolo7/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading and preprocessing the data

In [2]:
df=pd.read_excel(r"/home/runcandel/Desktop/pclubsectask/nlp/paragraphs.xlsx")
df=df.drop_duplicates(subset=['para'])
df=df.drop(columns=['idig'])
df.tail()

,topic,para
75041,2008_Summer_Olympics_torch_relay,"In Japan, the Mayor of Nagano, Shoichi Washiza..."
75046,2008_Summer_Olympics_torch_relay,The Olympic Flame is supposed to remain lit fo...
75048,2008_Summer_Olympics_torch_relay,A union planned to protest at the relay for be...
75049,2008_Summer_Olympics_torch_relay,"Chinese media have also reported on Jin Jing, ..."
75051,2008_Summer_Olympics_torch_relay,Two additional teams of 40 attendants each wil...


In [3]:
df.describe()

,topic,para
count,15557,15556
unique,362,15556
top,New_York_City,"Zhejiang (help·info), formerly romanized as C..."
freq,146,1


##### Creating a function to preprocess data

In [4]:
def pp(st):
    st=re.sub(r'[^\w\s]',"",st)
    stemm=PorterStemmer()
    st=st.split()
    st=[x for x in st if not x in stpwrds]
    st=[stemm.stem(x) for x in st]
    return " ".join(st)
    

##### Creating function check similarity bw texts by checking number of common words

In [5]:
def simcheck(a,b):
    q=0
    for i in a:
        if i in b:
            q+=1
    return q 


def mostsim(query,seq):
    l=[0,0,0,0,0]
    k=[0,0,0,0,0]
    for i in range(len(seq)):
        a=simcheck(query,seq[i])
        if a>min(k):
            for j in range(5):
                if k[j]<a:
                    l.insert(j,i)
                    k.insert(j,a)
                    l.pop(5)
                    k.pop(5)
                    break
    return l

##### created a list named data with all the paragraphs

In [6]:
data=df['para'].astype("str").to_list()

##### created a tokenizer and created dataf with processed data and seq with the tokenized sequences

In [7]:
tekken= Tokenizer()

tekken.fit_on_texts(data)
dataf=[]
for i in range(len(data)):
    dataf.append(pp(data[i]))

seq=tekken.texts_to_sequences(dataf)
print(seq[:10])
print(tekken.word_index['japanese'])





[[3365, 205, 41260, 290, 1845, 271, 3365, 959, 15254, 10467, 101, 24706, 1911, 16746, 118, 8242, 103, 129, 129, 271, 372, 1145, 4115, 30378, 239, 547], [1, 217, 30379, 327, 41261, 41262, 41263, 323, 217, 30380, 327, 2522, 785, 7030, 622, 3931, 151, 7030, 1298, 19, 4453, 37173, 2008, 11644, 327, 2008, 30381, 158, 4630, 151, 1320, 30382, 237, 82, 327, 217, 30381, 7031, 41265, 11923, 262, 30383, 217, 224, 9223, 357, 41266, 16747, 16748], [3365, 807, 41267, 30384, 6, 419, 990, 211, 995, 807, 218, 327, 450, 105, 8544, 30385, 30384, 1761, 18622], [1, 121, 156, 3365, 110, 4056, 14075, 455, 126, 3078, 341, 536, 121, 7887, 74, 9223, 1, 273, 9223, 159, 1371, 392, 550, 1616, 3681, 123, 17684, 273, 9223, 274, 3365, 41271, 1573, 706, 12851, 14075, 2495, 15255, 169, 2240, 273, 9223, 6609, 3645, 273, 101, 209, 271, 622, 1645, 392, 1616, 3681, 123, 13145, 1, 869, 9223, 41273, 41274, 1753, 41275, 41276, 5780, 869, 820, 9223, 392, 2584, 101, 271, 3366, 271, 41277, 21152, 146, 16749, 341, 1, 869, 450, 92

In [14]:
ques="japan nagano mayor"
lis=[ques]
qt=tekken.texts_to_sequences(lis)
print(qt)
print(mostsim(qt[0],seq))
l=mostsim(qt[0],seq)
for i in mostsim(qt[0],seq):
    print(data[i])

def freq(ques,seq):
    lis=[ques]
    qt=tekken.texts_to_sequences(lis)
    # print(qt)
    # print(mostsim(qt[0],seq))
    l=mostsim(qt[0],seq)
    for i in mostsim(qt[0],seq):
        print(data[i])
    return l


[[547, 13269, 1665]]
[15552, 374, 15528, 0, 14]
In Japan, the Mayor of Nagano, Shoichi Washizawa said that it has become a "great nuisance" for the city to host the torch relay prior to the Nagano leg. Washizawa's aides said the mayor's remark was not criticism about the relay itself but about the potential disruptions and confusion surrounding it.　A city employee of the Nagano City Office ridiculed the protests in Europe, he said "They are doing something foolish", in a televised interview. Nagano City officially apologized later and explained what he had wanted to say was "Such violent protests were not easy to accept". Also citing concerns about protests as well as the recent violence in Tibet, a major Buddhist temple in Nagano cancelled its plans to host the opening stage of the Olympic torch relay, this temple was vandalised by an un-identified person the day after in apparent revenge,
Finally, in the 1990s, Internet Protocol-based videoconferencing became possible, and more effic

### Trying the same above approach but this time using tf-idf vectors and cosine similarity index

In [15]:
tfidf_vectorizer = TfidfVectorizer()

ques="japan nagano mayor"
dataf.append(pp(ques))
tfidf_matrix = tfidf_vectorizer.fit_transform(dataf)
cssn=cosine_similarity(tfidf_matrix)
dataf.pop(-1)

# feature_names = tfidf_vectorizer.get_feature_names_out()
print(cssn[-1])

[0.0477435 0.        0.        ... 0.        0.        1.       ]


In [ ]:
l2=[0,0,0,0,0]
k=[0,0,0,0,0]
for i in range(len(cssn[-1])-1):
    if cssn[-1,i]>min(k):
        for j in range(5):
            if k[j]<cssn[-1,i]:
                l2.insert(j,i)
                k.insert(j,cssn[-1,i])
                l2.pop(5)
                k.pop(5)
                break
print(l2)

[15552, 5225, 4566, 13608, 14843]


In [ ]:
for i in l2:
    print(data[i])

In Japan, the Mayor of Nagano, Shoichi Washizawa said that it has become a "great nuisance" for the city to host the torch relay prior to the Nagano leg. Washizawa's aides said the mayor's remark was not criticism about the relay itself but about the potential disruptions and confusion surrounding it.　A city employee of the Nagano City Office ridiculed the protests in Europe, he said "They are doing something foolish", in a televised interview. Nagano City officially apologized later and explained what he had wanted to say was "Such violent protests were not easy to accept". Also citing concerns about protests as well as the recent violence in Tibet, a major Buddhist temple in Nagano cancelled its plans to host the opening stage of the Olympic torch relay, this temple was vandalised by an un-identified person the day after in apparent revenge,
Patience Latting was elected Mayor of Oklahoma City in 1971, becoming the city's first female mayor. Latting was also the first woman to serve a

##### making a function for the same

#### getting the question answer model to work

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

In [ ]:
def answer(l,ques,model,tokenizer):
    for i in range(len(l)):
        context = data[l[i]]
        inputs = tokenizer(ques, context, return_tensors="pt")
        outputs = model(**inputs)
        start_position = outputs.start_logits[0].argmax()
        end_position = outputs.end_logits[0].argmax()
        answer_ids = inputs["input_ids"][0][start_position:end_position]
        return tokenizer.decode(answer_ids)
ques='who was japan nagano mayor'

print("Answer:")
print(answer(l,ques,model,tokenizer))

Answer:
 Shoichi Washiz


In [ ]:
l=[10807, 14422, 10815, 10811, 14406]

In [ ]:
import nltk
from rake_nltk import Rake

In [ ]:
r=Rake()
r.extract_keywords_from_text(data[0])
print(r.get_phrases_with_scores())

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/runcandel/nltk_data'
    - '/home/runcandel/anaconda3/envs/yolo7/nltk_data'
    - '/home/runcandel/anaconda3/envs/yolo7/share/nltk_data'
    - '/home/runcandel/anaconda3/envs/yolo7/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [ ]:
import spacy
mod=spacy.load("en_core_web_lg")

In [ ]:
def simspa(ques,para):
    a=mod(ques)
    l=[0,0,0,0,0]
    k=[0,0,0,0,0]
    for i in range(len(para)):
        b=mod(para[i])
        h=a.similarity(b)
        if h>min(k):
            for j in range(5):
                if k[j]<h:
                    l.insert(j,i)
                    k.insert(j,h)
                    l.pop(5)
                    k.pop(5)
                    break
    for i in l:
        print(data[i])
    return l


In [ ]:
print(simspa("How many times was the FA cup playing in the Oval ?",data))

Winners receive the FA Cup trophy, of which there have been two designs and five actual cups; the latest is a 2014 replica of the second design, introduced in 1911. Winners also qualify for European football and a place in the FA Community Shield match. Arsenal are the current holders, having beaten Aston Villa 4–0 in the 2015 final to win the cup for the second year in a row. It was their 12th FA Cup title overall, making Arsenal the FA Cup's most successful club ahead of Manchester United on 11.
Arsenal Ladies are the women's football club affiliated to Arsenal. Founded in 1987, they turned semi-professional in 2002 and are managed by Clare Wheatley. Arsenal Ladies are the most successful team in English women's football. In the 2008–09 season, they won all three major English trophies – the FA Women's Premier League, FA Women's Cup and FA Women's Premier League Cup, and, as of 2009, were the only English side to have won the UEFA Women's Cup, having done so in the 2006–07 season as 